In [ ]:
import os
from ultralytics import YOLO

model_path = '../Models/best_model_yolo.pt'
model = YOLO(model_path)

print(f"Modello caricato da: {model_path}")

In [2]:
import cv2

def predict_webcam(camera_id=0, conf_threshold=0.3):
    """
    Esegue l'inferenza in tempo reale utilizzando la webcam
    
    Args:
        camera_id: ID della webcam (di solito 0 per la webcam integrata)
        conf_threshold: Soglia di confidenza per le predizioni
    """
    
    cap = cv2.VideoCapture(camera_id)
    
    if not cap.isOpened():
        print(f"Errore nell'apertura della webcam con ID: {camera_id}")
        return
    
    print("Inferenza in tempo reale avviata. Premi 'q' per uscire.")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        results = model(frame, conf=conf_threshold)
        result_frame = results[0].plot()
        
        cv2.imshow('Rilevamento Pistole in Tempo Reale', result_frame)
        
        # Esci se viene premuto 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    print("Inferenza in tempo reale terminata.")

In [3]:
import matplotlib.pyplot as plt

def predict_folder(folder_path, conf_threshold=0.3):
    """
    Esegue l'inferenza su tutte le immagini in una cartella e mostra i risultati
    
    Args:
        folder_path: Percorso della cartella contenente le immagini
        conf_threshold: Soglia di confidenza per le predizioni
    """
    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp')
    
    image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(valid_extensions)]
    
    if not image_files:
        print(f"Nessuna immagine trovata in: {folder_path}")
        return
    
    print(f"Trovate {len(image_files)} immagini. Inizio inferenza...\n")
    
    for img_file in image_files:
        img_path = os.path.join(folder_path, img_file)
        
        img = cv2.imread(img_path)
        results = model(img, conf=conf_threshold, verbose=False)
        result_img = results[0].plot()
        
        result_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
        
        plt.figure(figsize=(10, 8))
        plt.imshow(result_img)
        plt.title(f'Predizione - {img_file}')
        plt.axis('off')
        plt.show()
        
        boxes = results[0].boxes
        if len(boxes) > 0:
            for box in boxes:
                conf = box.conf.item()
                print(f"- Rilevamento con confidenza: {conf:.2f}")
        else:
            print("- Nessun rilevamento")
        print()
    
    print("Inferenza completata.")

In [ ]:
predict_folder(folder_path='demo_images', conf_threshold=0.5)
predict_webcam(camera_id=0, conf_threshold=0.5)